In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# TensorFlow and keras
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from keras.models import Sequential
# Helper libraries
import os
from glob import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
import cv2
from keras.models import Model
from keras.layers import add
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers.merge import concatenate
from keras.utils import plot_model
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate
)
import imgaug as ia
from imgaug import augmenters as iaa

Using TensorFlow backend.


In [ ]:
class DataGenerator(keras.utils.Sequence):
    #'Generates data for Keras'
    def __init__(self, root_dir, task, plane, augment=False, train=True):
        super().__init__()
        #'Initialization'
        self.root_dir = root_dir
        self.task = task
        self.plane = plane
        self.train = train
        self.augment = augment

        if self.train:
            self.folder_path = self.root_dir + 'train/{0}/'.format(plane)
            self.record = pd.read_csv(self.root_dir + 'train-{0}.csv'.format(task),
                                header=None,
                                names=['id', 'label'],
                                dtype={'id': str, 'label': np.int64})
        else:
            self.folder_path = self.root_dir + 'valid/{0}/'.format(plane)
            self.record = pd.read_csv(self.root_dir + 'valid-{0}.csv'.format(task),
                                header=None,
                                names=['id', 'label'],
                                dtype={'id': str, 'label': np.int64})
        self.paths = [self.folder_path + filename + '.npy' for filename in self.record['id'].tolist()]
        self.labels = self.record['label'].tolist()
        
    def augmentor(self, images):
        sometimes = lambda aug: iaa.Sometimes(0.5, aug)
        seq = iaa.Sequential(
              [
              # apply the following augmenters to most images
              iaa.Fliplr(0.5),  # horizontally flip 50% of all images
              iaa.Flipud(0.2),  # vertically flip 20% of all images
              sometimes(iaa.Affine(
                scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
                # scale images to 80-120% of their size, individually per axis
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
                # translate by -20 to +20 percent (per axis)
                rotate=(-10, 10),  # rotate by -45 to +45 degrees
                shear=(-5, 5),  # shear by -16 to +16 degrees
                order=[0, 1],
                # use nearest neighbour or bilinear interpolation (fast)
                cval=(0, 255),  # if mode is constant, use a cval between 0 and 255
                mode=ia.ALL
                # use any of scikit-image's warping modes (see 2nd image from the top for examples)
              ))
              ],
              random_order=True
        )
        return seq.augment_images(images)  

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        path = np.load(self.paths[index])
        batch = path.shape[0]

        c = np.empty((batch, 256, 256, 3))
        label = np.empty((batch), dtype=int)
        for y in range (batch):
            X = path[y]
            X = np.array(X, dtype=K.floatx())
            X = X / 255.0
            X = np.expand_dims(X, axis=0)
            X = np.stack((X,)*3, axis=3)
            c[y] = X
            label[y] = self.labels[index]
        if self.augment == True:
            c = self.augmentor(c)
        return c, keras.utils.to_categorical(label, num_classes=2)